In [1]:
import os
import random

import mlflow
import pandas as pd

In [2]:
# autimatically parsed from environment varabie
print(mlflow.tracking.get_tracking_uri())
print(os.environ["MLFLOW_TRACKING_URI"])

http://mlflow:5000
http://mlflow:5000


## Sample experiment

In [3]:
mlflow.set_experiment("mlflow_experiments")

<Experiment: artifact_location='/data/artifacts/1', experiment_id='1', lifecycle_stage='active', name='mlflow_experiments', tags={}>

In [4]:
# starting and tearing down the experiment
with mlflow.start_run():
    mlflow.log_param("name","mlflow_test")
    mlflow.log_param("category","algorithm")
    mlflow.log_param("type","classification")
    for i in range(5):
        mlflow.log_metric("i", i, step=i)
    mlflow.log_artifact("02_docker_stack_example.ipynb")

## Update with your own algorithms

In [5]:
class RandomPredictor(mlflow.pyfunc.PythonModel):
    def __init__(self):
        pass

    def predict(self, context, model_input):
        return model_input.apply(lambda column: random.randint(0,1))

In [6]:
# Construct and save the model
model_path = "models/randomized_model"
r = RandomPredictor()
mlflow.pyfunc.save_model(path=model_path, python_model=r)

/opt/conda/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [7]:
# load and used the saved model
loaded_model = mlflow.pyfunc.load_model(model_path)

In [8]:
print(eval(loaded_model.metadata.to_json()))

{'utc_time_created': '2022-09-18 12:45:33.244927', 'flavors': {'python_function': {'cloudpickle_version': '2.2.0', 'env': 'conda.yaml', 'loader_module': 'mlflow.pyfunc.model', 'python_model': 'python_model.pkl', 'python_version': '3.10.6'}}, 'model_uuid': '2f2430f303e14334b284a25a9df9fb79', 'mlflow_version': '1.28.0'}


In [9]:
model_inputs = pd.DataFrame([range(10)])

random_predictor = RandomPredictor()
mlflow.set_experiment("stockpred_experiment_days_up")
with mlflow.start_run():
    model_output = loaded_model.predict(model_inputs)
    
    mlflow.log_metric("Days up", model_output.sum())
    mlflow.log_artifact("02_docker_stack_example.ipynb")

2022/09/18 12:45:34 INFO mlflow.tracking.fluent: Experiment with name 'stockpred_experiment_days_up' does not exist. Creating a new experiment.
